<a href="https://colab.research.google.com/github/rasmus-pagh/apx/blob/main/Vertex_cover.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import urllib.request
import os.path

class DataFile:

  url_prefix = 'https://rasmuspagh.net/data/'
  graph_files = ['routes.txt','petersen.txt','star.txt','clique.txt','cycles.txt','lotr.txt','karate.txt']

  def __init__(self, filename):
    if not os.path.isfile(filename):
      urllib.request.urlretrieve(self.url_prefix + filename, filename)  
    if not os.path.isfile(filename):
      raise ValueError('Unknown file: {filename}\nKnown files: {files}')
    else: 
      self.f = open(filename, "r")

  def __iter__(self):
    return self

  def __next__(self):
    line = self.f.readline()
    if line == '':
      raise StopIteration
    return [ x for x in line.rstrip('\n').split(' ') if x != '']

In [ ]:
import numpy as np
np.warnings.filterwarnings('ignore', category=np.VisibleDeprecationWarning)
from scipy.optimize import linprog
from scipy.sparse import coo_matrix
import re

class LinearProgram:

  term_re = re.compile('([-+]?)(\d*\.\d+|\d+)?\*?(\w+)')

  def __init__(self, objective_type = 'max'):
    self.objective_type = objective_type
    self.row_numbers = []
    self.column_numbers = []
    self.entry_weights = []
    self.bounds = []
    self.objective = []
    self.map_name_column = {}
    self.map_column_name = {}
    self.map_name_row = {}
    self.map_row_name = {}
    self.num_columns = 0
    self.num_rows = 0

  def column_number(self, column_name):
    if not column_name in self.map_name_column:
      self.map_name_column[column_name] = self.num_columns
      self.map_column_name[self.num_columns] = column_name
      self.num_columns += 1
    return self.map_name_column[column_name]

  def parse_expression(self, x):
    map_name_weight = {}
    for match in self.term_re.finditer(x.replace(" ", "")):
      if match.group(1) == '-':
        sign = -1
      else:
        sign = 1
      if match.group(2) is None:
        weight = 1.0
      else:
        weight = float(match.group(2))
      map_name_weight[match.group(3)] = sign * weight
    return map_name_weight

  def add_constraint(self, sparse_row, b, name = None):
    if isinstance(sparse_row, str):
      map_name_weight = self.parse_expression(sparse_row)
    else:
      map_name_weight = sparse_row
    self.bounds.append(b)
    for column_name in map_name_weight:
      self.row_numbers.append(self.num_rows)
      self.column_numbers.append(self.column_number(column_name))
      self.entry_weights.append(map_name_weight[column_name])
    if name is None:
      i = self.num_rows + 1
      while 'y'+str(i) in self.map_name_row: # Find unique row name
        i += 1
      name = 'y'+str(i)
    assert(name not in self.map_name_row)
    self.map_name_row[name] = self.num_rows
    self.map_row_name[self.num_rows] = name
    self.num_rows += 1

  def set_objective(self, sparse_objective):
    if isinstance(sparse_objective, str):
      sparse_objective = self.parse_expression(sparse_objective)
    for column_name in sparse_objective: # Ensure that all names map to columns
      self.column_number(column_name)
    self.objective = [sparse_objective.get(self.map_column_name[j], 0.0) for j in range(self.num_columns)]

  def to_string(self):
    A = coo_matrix((self.entry_weights, (self.row_numbers, self.column_numbers))).todense()
    if self.objective_type == 'max':
      return f'Maximize c x under A x <= b, x >= 0, where\nA={A}\nb={self.bounds}\nc={self.objective}'
    else:
      return f'Minimize b y under A y >= c, y >= 0, where\nA={A}\nb={self.objective}\nc={self.bounds}'
  
  def dual(self):
    if self.objective_type == 'max':
      res = LinearProgram('min')
    else:
      res = LinearProgram('max')    
    res.entry_weights = self.entry_weights.copy()
    res.row_numbers = self.column_numbers.copy()
    res.column_numbers = self.row_numbers.copy()
    res.bounds = self.objective.copy()
    res.objective = self.bounds.copy()
    res.map_name_column = self.map_name_row.copy()
    res.map_column_name = self.map_row_name.copy()
    res.map_name_row = self.map_name_column.copy()
    res.map_row_name = self.map_column_name.copy()
    res.num_columns = self.num_rows
    res.num_rows = self.num_columns
    return res

  def solve(self):
    A = coo_matrix((self.entry_weights, (self.row_numbers, self.column_numbers))).todense()
    b = np.array(self.bounds)
    c = np.array(self.objective)
    if self.objective_type == 'max':
      sign = -1
    elif self.objective_type == 'min':
      sign = 1
    else:
      raise ValueError(f'Unknown objective type: {self.objective}')
    res = linprog(sign * c, A_ub=-sign*A, b_ub=-sign*b, options={'sym_pos': False, 'lstsq': True})
    solution_dict = {}
    for column_name in self.map_name_column:
      solution_dict[column_name] = sign * res.x[self.map_name_column[column_name]]
    return sign * res.fun, solution_dict

In [ ]:
mylp = LinearProgram('max')
mylp.add_constraint({"x1": 4, "x2": 1}, 2)
mylp.add_constraint("x1 + 2*x2", 1)
mylp.set_objective({"x1": 1, "x2": 1})
print(mylp.to_string())
print(mylp.solve())
dual = mylp.dual()
print(dual.to_string())
print(dual.solve())
print(dual.dual().to_string())
print(dual.dual().solve())


Maximize c x under A x <= b, x >= 0, where
A=[[4. 1.]
 [1. 2.]]
b=[2, 1]
c=[1, 1]
(0.7142857142880961, {'x1': -0.4285714285729371, 'x2': -0.285714285715159})
Minimize b y under A y >= c, y >= 0, where
A=[[4. 1.]
 [1. 2.]]
b=[2, 1]
c=[1, 1]
(0.714285714291272, {'y1': 0.14285714285772957, 'y2': 0.4285714285758128})
Maximize c x under A x <= b, x >= 0, where
A=[[4. 1.]
 [1. 2.]]
b=[2, 1]
c=[1, 1]
(0.7142857142880961, {'x1': -0.4285714285729371, 'x2': -0.285714285715159})


In [ ]:
for filename in DataFile.graph_files:
  graph = DataFile(filename)
  vertex_cover_lp = LinearProgram('min')
  objective = {}
  for (u,v) in graph:
    vertex_cover_lp.add_constraint({u: 1, v: 1}, 1)
    objective[u] = 1.0
    objective[v] = 1.0

  vertex_cover_lp.set_objective(objective)
  value, solution = vertex_cover_lp.solve()
  rounded_value, rounded_solution = 0, {}
  for x in solution:
    r = int(np.round(solution[x] + 1e-10))
    rounded_solution[x] = r
    rounded_value += r

  print(f'Graph: {filename}')
  print(f'LP optimal value: {value}\n{solution}')
  print(f'Rounded LP value: {rounded_value}\n{rounded_solution}')


Graph: routes.txt
LP optimal value: 5.000000000001148
{'JFK': 0.5000000000000834, 'MCO': 0.50000000000015, 'ORD': 0.500000000000015, 'DEN': 0.5000000000001982, 'HOU': 0.5000000000000092, 'DFW': 0.5000000000002384, 'PHX': 0.5000000000000608, 'ATL': 0.5000000000002527, 'LAX': 0.5000000000001774, 'LAS': 0.4999999999999635}
Rounded LP value: 10
{'JFK': 1, 'MCO': 1, 'ORD': 1, 'DEN': 1, 'HOU': 1, 'DFW': 1, 'PHX': 1, 'ATL': 1, 'LAX': 1, 'LAS': 1}
Graph: petersen.txt
LP optimal value: 5.000000000881439
{'A': 0.5000000000891484, 'B': 0.500000000088124, 'C': 0.5000000000875104, 'D': 0.5000000000887319, 'E': 0.5000000000875057, '1': 0.5000000000875143, '3': 0.5000000000885372, '5': 0.5000000000883218, '2': 0.5000000000879159, '4': 0.5000000000881305}
Rounded LP value: 10
{'A': 1, 'B': 1, 'C': 1, 'D': 1, 'E': 1, '1': 1, '3': 1, '5': 1, '2': 1, '4': 1}
Graph: star.txt
LP optimal value: 1.0000000000180407
{'1': 0.9999999999979554, '2': 2.5106967315632556e-12, '3': 2.5106967327543957e-12, '4': 2.5106